In [ ]:
import requests

# Substitua pelos seus valores
access_token = 'SEU_ACCESS_TOKEN'
video_path = 'caminho/para/seu/video.mp4'
title = 'Título do seu vídeo'
privacy_level = 'PUBLIC_TO_EVERYONE'  # Opções: PUBLIC_TO_EVERYONE, MUTUAL_FOLLOW_FRIENDS, FOLLOWER_OF_CREATOR, SELF_ONLY

# Inicializar a postagem
init_url = 'https://open.tiktokapis.com/v2/post/publish/video/init/'
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json; charset=UTF-8'
}
data = {
    'post_info': {
        'title': title,
        'privacy_level': privacy_level
    },
    'source_info': {
        'source': 'FILE_UPLOAD',
        'video_size': os.path.getsize(video_path),
        'chunk_size': 10000000,  # 10MB por chunk
        'total_chunk_count': (os.path.getsize(video_path) // 10000000) + 1
    }
}
response = requests.post(init_url, json=data, headers=headers)
if response.status_code != 200:
    print(f'Erro na inicialização: {response.text}')
    exit()

upload_url = response.json()['data']['upload_url']
publish_id = response.json()['data']['publish_id']

# Enviar o vídeo em partes
with open(video_path, 'rb') as video_file:
    chunk_size = 10000000  # 10MB por chunk
    for i in range(0, os.path.getsize(video_path), chunk_size):
        chunk = video_file.read(chunk_size)
        headers = {
            'Content-Range': f'bytes {i}-{i+len(chunk)-1}/{os.path.getsize(video_path)}',
            'Content-Type': 'video/mp4'
        }
        response = requests.put(upload_url, headers=headers, data=chunk)
        if response.status_code != 200:
            print(f'Erro no upload do chunk {i//chunk_size + 1}: {response.text}')
            exit()

# Verificar o status da postagem
status_url = 'https://open.tiktokapis.com/v2/post/publish/status/fetch/'
data = {'publish_id': publish_id}
response = requests.post(status_url, json=data, headers=headers)
if response.status_code == 200:
    print('Vídeo publicado com sucesso!')
else:
    print(f'Erro ao verificar o status: {response.text}')
